In [0]:
%sql
CREATE OR REFRESH MATERIALIZED VIEW IDENTIFIER (:GOLD_CATALOG || "." || :GOLD_SCHEMA || "." || :FCT_DAILY_PROFILE_STATS_TABLE) AS

-- 1: select daily stats for individual posts
SELECT
  concat(
    i.post_publish_date, 
    ": ", 
    regexp_extract(p.link, r'.*_(.*)-activity.*')
  ) AS post_id,
  i.post_publish_date,
  p.post_publish_timestamp,
  impressions,
  coalesce(engagements, 0) as engagements,
  i.analytics_date,
  DATEDIFF(i.analytics_date, i.post_publish_date) AS days_since_post,
  coalesce(impressions - LAG(impressions, 1) OVER (
    PARTITION BY i.post_url ORDER BY i.analytics_date
  )) AS change_in_impressions,
  coalesce(engagements - LAG(engagements, 1) OVER (
    PARTITION BY i.post_url ORDER BY i.analytics_date
  )) AS change_in_engagements,
  p.title,
  p.content
FROM
  IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_IMPRESSIONS_TABLE) i
  LEFT JOIN IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_ENGAGEMENTS_TABLE) e
    ON i.post_url = e.post_url
    AND i.analytics_date = e.analytics_date
  LEFT JOIN silver.linkedin.posts p
    ON i.post_url = p.post_url
WHERE i.post_url != "others"

UNION ALL

-- 2: select daily stats for posts outside of top 50 for the day
SELECT
  "others" AS post_id,
  NULL AS post_publish_date,
  NULL AS post_publish_timestamp,
  impressions,
  coalesce(engagements, 0) AS engagements,
  i.analytics_date,
  NULL AS days_since_post,
  NULL AS change_in_impressions,
  NULL AS change_in_engagements,
  NULL AS title,
  NULL AS content  
FROM 
  (
    SELECT
      impressions,
      analytics_date
    FROM
      IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_IMPRESSIONS_TABLE)
    WHERE post_url = "others"
  ) i 
  LEFT JOIN (
    SELECT
      engagements,
      analytics_date
    FROM
      IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_ENGAGEMENTS_TABLE)
    WHERE post_url = "others"
  ) e
  ON i.analytics_date = e.analytics_date
  
UNION ALL

-- 3: select total stats
SELECT
  "total" AS post_id,
  NULL AS post_publish_date,
  NULL AS post_publish_timestamp,
  t.impressions,
  t.engagements,
  t.date AS analytics_date,
  NULL AS days_since_post,
  coalesce(
    t.impressions - prev_t.impressions
  ) AS change_in_impressions,
  coalesce(
    t.engagements - prev_t.engagements
  ) AS change_in_engagements,
  NULL AS title,
  NULL AS content
FROM
  IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE) t
  LEFT JOIN IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE) prev_t
    ON prev_t.date = DATE_SUB(t.date, 1);

In [0]:
%sql
CREATE OR REFRESH MATERIALIZED VIEW IDENTIFIER (:GOLD_CATALOG || "." || :GOLD_SCHEMA || "." || :FCT_DAILY_POST_STATS_TABLE) AS
SELECT
  t.date AS analytics_date,
  f.new_followers,
  t.impressions,
  t.engagements,
  coalesce(
    t.impressions - prev_t.impressions
  ) AS change_in_impressions,
  coalesce(
    t.engagements - prev_t.engagements
  ) AS change_in_engagements
FROM
  IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE) t
  LEFT JOIN IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE) prev_t
    ON prev_t.date = DATE_SUB(t.date, 1)
  LEFT JOIN IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_FOLLOWERS_TABLE) f
    ON f.date = t.date
ORDER BY t.date ASC;

In [0]:
%sql
CREATE OR REFRESH MATERIALIZED VIEW IDENTIFIER (:GOLD_CATALOG || "." || :GOLD_SCHEMA || "." || :DIM_DATE_TABLE) AS
SELECT
  date,
  DAY(date) AS day,
  MONTH(date) AS month,
  YEAR(date) AS year,
  WEEKOFYEAR(date) AS week,
  WEEKDAY(date) AS weekday,
  DATE_FORMAT(date, 'E') AS weekday_str,
  WEEKDAY(date) >= 5 AS is_weekend
FROM (
  SELECT date 
  FROM IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_TOTALS_TABLE)
  UNION 
  SELECT post_publish_date AS date 
  FROM IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_POSTS_TABLE)
)
ORDER BY date ASC

In [0]:
%sql
CREATE OR REFRESH MATERIALIZED VIEW IDENTIFIER (:GOLD_CATALOG || "." || :GOLD_SCHEMA || "." || :DIM_TIMESTAMP_TABLE) AS
SELECT
  timestamp,
  sg_timestamp,
  DATE(timestamp) AS utc_date,
  HOUR(timestamp) AS utc_hour,
  MINUTE(timestamp) AS utc_minute,
  DATE(sg_timestamp) AS sgt_date,
  HOUR(sg_timestamp) AS sgt_hour,
  MINUTE(sg_timestamp) AS sgt_minute
FROM (
  SELECT DISTINCT 
    post_publish_timestamp AS timestamp,
    from_utc_timestamp(
      post_publish_timestamp, 'Asia/Singapore'
    ) AS sg_timestamp
  FROM IDENTIFIER (:SILVER_CATALOG || "." || :SILVER_SCHEMA || "." || :SILVER_POSTS_TABLE)
  WHERE post_publish_timestamp IS NOT NULL
)
ORDER BY timestamp ASC